# Lab | Connecting Python to SQL

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass #to get password without showing input
password = getpass.getpass()

 ········


1. Establish a connection between Python and the Sakila database.

In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [3]:
def rentals_month(engine, month:int, year:int)-> pd.DataFrame:
    """
    The function executes a SQL query to retrieve the rental data for the specified 
    month and year from the rental table in the Sakila database, and returns 
    it as a pandas DataFrame.

    Input
    `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	`month`: an integer representing the month for which rental data is to be retrieved.
	`year`: an integer representing the year for which rental data is to be retrieved.

    Output
    pd.DataFrame
    """
    engine = engine
    
    query = "SELECT * \
            FROM rental \
            WHERE MONTH(rental_date) = {} and YEAR(rental_date) = {} \
            ".format(month,year)
    
    data = pd.read_sql_query(query,engine)

    return data

In [4]:
df = rentals_month(engine, 5, 2005)
df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".


	*Hint: Consider making use of pandas [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)*

In [5]:
def rental_count_month(engine, month:int, year:int)->pd.DataFrame:
    """
    Returns a new dataframe containing the number of rentails made by each customer_id during the selected month and year

    Input
    engine: engine
    month: integer value of month
    year: integer value of year

    Output
    pd.DataFrame
    """
    df = rentals_month(engine, month, year)

    df_temp = df.copy()

    df_temp = df_temp.groupby('customer_id')[['rental_id']].count()
    col_name = ['rental_'+str(month)+'_'+str(year)]
    df_temp.columns = col_name
    df_temp.reset_index(inplace = True)

    return df_temp


In [6]:
rental_count_month(engine, 5, 2005)

,customer_id,rental_5_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [7]:
def compare_rentals(engine, first_month:int, second_month:int, first_year:int, second_year:int)->pd.DataFrame:
    """
    Creates a new DataFrame for the number of rentals by customer_id for two dates (month and year) and a new column with the difference
    in rentals between these two dates

    Input
    engine: engine
    first_month: integer value of first month to compare
    second_month: integer value of second month to compare
    first_year: integer value of first year
    second_year: integer value of second year

    Output
    pd.DataFrame

    """
    df1 = rental_count_month(engine,first_month,first_year)
    df1_temp = df1.copy()

    df2 = rental_count_month(engine,second_month, second_year)
    df2_temp = df2.copy()

    #merge dataframes
    compare_df = pd.merge(df1_temp,df2_temp, how='outer', on='customer_id', sort=True)
    compare_df = compare_df.fillna(0).astype(int)

    #create new difference column
    compare_df['difference'] = compare_df.iloc[:,2]-compare_df.iloc[:,1]

    return compare_df

In [8]:
test = compare_rentals(engine,5,6,2005,2005)
test

,customer_id,rental_5_2005,rental_6_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,4,0,6,6
4,5,3,5,2
...,...,...,...,...
593,595,1,2,1
594,596,6,2,-4
595,597,2,3,1
596,598,0,1,1


In [9]:
test.isna().sum()

customer_id      0
rental_5_2005    0
rental_6_2005    0
difference       0
dtype: int64